**Fake News Classificer Using Bidirectional LSTM**

In [ ]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/train.csv.zip")

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df.shape

(20800, 5)

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [ ]:
## Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1660, 4786, 4910, 3421, 464, 4044, 3392, 222, 232, 1683],
 [4623, 4072, 4538, 2510, 3490, 2404, 2846],
 [53, 1378, 1453, 4771],
 [947, 4513, 4189, 2816, 892, 3041],
 [4340, 3490, 4153, 2069, 1888, 354, 3490, 2137, 3359, 3632],
 [3759,
  2217,
  1323,
  4850,
  319,
  968,
  429,
  1878,
  2809,
  4799,
  2546,
  2841,
  1927,
  4198,
  2846],
 [1928, 75, 3920, 3358, 1703, 191, 2695, 3747, 972, 1132, 4576],
 [2167, 445, 2233, 843, 1629, 4953, 968, 9, 972, 1132, 4576],
 [1377, 1489, 2566, 374, 3401, 1292, 2826, 4545, 968, 364],
 [3508, 2388, 516, 2514, 147, 556, 4911, 3785],
 [2111, 800, 2720, 2599, 636, 4083, 2295, 305, 2272, 4699, 611],
 [2816, 4804, 464, 1292, 968, 1629],
 [294, 2983, 4628, 1061, 1910, 3457, 2218, 3184, 2299],
 [23, 1320, 1027, 1677, 320, 2665, 2964, 972, 1132, 4576],
 [1771, 1487, 1255, 2279, 1410, 972, 1132, 4576],
 [603, 3124, 2848, 4388, 4707, 3568, 1443, 4710, 3365, 4441],
 [1875, 3618, 4072],
 [4219, 252, 3156, 4141, 968, 1011, 533, 2846],
 [1818, 3814, 4538, 

Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  222,  232, 1683],
       [   0,    0,    0, ..., 3490, 2404, 2846],
       [   0,    0,    0, ..., 1378, 1453, 4771],
       ...,
       [   0,    0,    0, ...,  972, 1132, 4576],
       [   0,    0,    0, ..., 3537, 4459,   31],
       [   0,    0,    0, ..., 1803,  868,  248]], dtype=int32)

In [ ]:
##Creating The Model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary-crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)



In [ ]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 5s 13ms/step - loss: 4.4158 - accuracy: 0.7142 - val_loss: 3.1562 - val_accuracy: 0.7430
Epoch 2/10
383/383 [==============================] - 2s 6ms/step - loss: 4.3000 - accuracy: 0.7139 - val_loss: 3.8759 - val_accuracy: 0.7190
Epoch 3/10
383/383 [==============================] - 1s 3ms/step - loss: 3.8910 - accuracy: 0.7202 - val_loss: 2.3744 - val_accuracy: 0.7425
Epoch 4/10
383/383 [==============================] - 1s 3ms/step - loss: 2.9793 - accuracy: 0.7211 - val_loss: 2.4206 - val_accuracy: 0.7287
Epoch 5/10
383/383 [==============================] - 1s 3ms/step - loss: 3.1362 - accuracy: 0.7256 - val_loss: 2.2121 - val_accuracy: 0.7259
Epoch 6/10
383/383 [==============================] - 2s 4ms/step - loss: 3.6919 - accuracy: 0.7221 - val_loss: 3.1843 - val_accuracy: 0.7379
Epoch 7/10
383/383 [==============================] - 1s 3ms/step - loss: 2.7410 - accuracy: 0.7285 - val_loss: 3.3186 - val_accuracy: 0.7051
Epoch

In [ ]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [ ]:
import numpy as np
y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[2860,  559],
       [ 931, 1685]])

In [ ]:
print(accuracy_score(y_test,y_pred))

0.7531068765534383
